In [16]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
# Cargar los archivos CSV
df_maestro = pd.read_csv("L_STMaestroPrd.csv")  # Tu archivo maestro
df_proveedor = pd.read_csv("prueba.csv")  # El archivo de proveedor

C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_15904\2345330016.py:2: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df_maestro = pd.read_csv("L_STMaestroPrd.csv")  # Tu archivo maestro


In [18]:
import pandas as pd
import re
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Copias de los DataFrames
df_productos = pd.read_csv("L_STMaestroPrd.csv")
copy_dfProductos = df_productos.loc[:, df_productos.notna().any()].copy()  # Eliminar columnas vacías

# Este modulo estandariza los csv que provienen del proveedor.
# Función para procesar el caso de "varios productos en una fila"
def procesar_varios_productos(df):
    for column in df.columns:
        df[column] = df[column].apply(lambda x: x.split('\n') if isinstance(x, str) else ([x] if pd.notna(x) else [np.nan]))
    df_exploded_list = []
    for _, row in df.iterrows():
        row_lists = [row[col] if isinstance(row[col], list) else [row[col]] for col in df.columns]
        max_len = max(len(lst) for lst in row_lists)
        row_lists = [lst + [None] * (max_len - len(lst)) for lst in row_lists]
        exploded_row = pd.DataFrame(row_lists).T
        exploded_row.columns = df.columns
        df_exploded_list.append(exploded_row)
    return pd.concat(df_exploded_list, ignore_index=True)

# Función para procesar el caso de "un producto por fila"
def procesar_un_producto(df):
    for col in df.columns:
        df[col] = df[col].apply(lambda x: x.replace('\r\n', ' ').replace('\n', ' ') if isinstance(x, str) else x)
    return df

# Función principal para decidir el módulo a usar
def procesar_csv(file_path):
    df = pd.read_csv(file_path)
    contiene_saltos = df.iloc[:, 0].apply(lambda x: '\n' in x if isinstance(x, str) else False).any()
    if contiene_saltos:
        print("Usando el módulo: VARIOS PRODUCTOS EN UNA FILA")
        return procesar_varios_productos(df)
    else:
        print("Usando el módulo: UN PRODUCTO POR FILA")
        return procesar_un_producto(df)

# Cargar el CSV del proveedor
ruta_csv = "prueba2.csv"
df_resultado_PROVEEDOR = procesar_csv(ruta_csv)

# Función de limpieza básica de texto para la columna de descripciones
def clear_column(column):
    columna_limpia = re.sub(r'\b(\d+|mg|comp\.?|blisters?|estuche|x|ml|unidades|marca|botella|caja|referencia|cm|g|m|f\.a\.x|'
                            r'jbe\.|sol\.|env\.|lata|vial|pvo\.|sobres|gotero|cápsulas|frascos?|blister|bot\.|mcg|sp|p\.b\.)\b',
                            '', column, flags=re.IGNORECASE)
    columna_limpia = re.sub(r'\n', ' ', columna_limpia)
    columna_limpia = re.sub(r'[:\.\-,;()]', '', columna_limpia)
    columna_limpia = re.sub(r'\s+', ' ', columna_limpia).strip()
    return columna_limpia

# Limpiamos las descripciones de los productos
df_resultado_PROVEEDOR['descripcion_limpia'] = df_resultado_PROVEEDOR['DESCRIPCIÓN'].fillna('').apply(clear_column)

# Función para filtrar productos por laboratorio
def laboratorio_input_df(df, columna):
    input_name = input("Ingrese el nombre del laboratorio: ").strip()
    tablas_coincidencias = df[df[columna].str.contains(input_name, case=False, na=False)].copy()
    if not tablas_coincidencias.empty:
        print(f"Se encontraron {len(tablas_coincidencias)} productos coincidentes del laboratorio '{input_name}'.")
        tablas_coincidencias['descrip1_limpia'] = tablas_coincidencias['descrip1'].fillna('').apply(clear_column)
        return tablas_coincidencias
    else:
        print(f"No se encontraron productos del laboratorio '{input_name}'.")
        return None

column_name_laboratorio = "atrib0"
laboratorio_coincidencias_df = laboratorio_input_df(df_productos, column_name_laboratorio)

# Instanciamos el modelo de SentenceTransformer para obtener embeddings de texto
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Función para obtener embeddings de las descripciones
def get_embeddings(descriptions):
    return model.encode(descriptions.tolist())

# Comparar descripciones de productos utilizando embeddings
if laboratorio_coincidencias_df is not None:
    proveedor_embeddings = get_embeddings(df_resultado_PROVEEDOR['descripcion_limpia'])
    laboratorio_embeddings = get_embeddings(laboratorio_coincidencias_df['descrip1_limpia'])

    matches = []

    for i, prov_row in df_resultado_PROVEEDOR.iterrows():
        prov_desc = prov_row['descripcion_limpia']

        # Calculamos la similitud de coseno entre el producto proveedor y los productos del laboratorio
        cosine_similarities = cosine_similarity([proveedor_embeddings[i]], laboratorio_embeddings)

        best_match_index = cosine_similarities.argmax()  # El índice de la mejor coincidencia
        best_match_score = cosine_similarities.max()  # El puntaje de similitud

        match_info = prov_row.to_dict()
        match_info.update(laboratorio_coincidencias_df.iloc[best_match_index].to_dict())
        match_info['similaridad'] = best_match_score
        matches.append(match_info)

    df_matches = pd.DataFrame(matches)

    # Reordenamos las columnas para que la columna 'similaridad' esté al inicio
    cols = ['similaridad'] + [col for col in df_matches.columns if col != 'similaridad']
    df_matches = df_matches[cols]

    print(f"\nCoincidencias encontradas: {len(df_matches[df_matches['similaridad'] > 0])}")
    print(df_matches)

    productos_sin_coincidencia = laboratorio_coincidencias_df[~laboratorio_coincidencias_df['descrip1_limpia'].isin(
        df_matches[df_matches['similaridad'] > 0]['descrip1_limpia']
    )]

    print(f"\nProductos del laboratorio sin coincidencias: {len(productos_sin_coincidencia)}")
    print(productos_sin_coincidencia)
else:
    print("No se encontraron coincidencias para el laboratorio especificado.")


C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_15904\1578396040.py:8: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df_productos = pd.read_csv("L_STMaestroPrd.csv")


Usando el módulo: VARIOS PRODUCTOS EN UNA FILA
Se encontraron 100 productos coincidentes del laboratorio 'ABBV'.

Coincidencias encontradas: 27
    similaridad Unnamed: 0                                 DESCRIPCIÓN  \
0      0.840433   20015888                        HUMIRA AC x 2 (20mg)   
1      0.766043   20005289  HUMIRA AC x 2 (40mg) HUMIRA AC x 1 (80 mg)   
2      0.352425   20021980                                        None   
3      0.481004   20062091                     RINVOQ 15MG RINVOQ 30MG   
4      0.528795   20069005                                 RINVOQ 45MG   
5      0.352425   20078725                                        None   
6      0.979447   20069017                             SKYRIZI SKYRIZI   
7      1.000000   20079636                                     SKYRIZI   
8      0.352425   20079635                                        None   
9      0.738401   20058578                            KALETRA SOLUCIÓN   
10     0.673247   20025520     KALETRA COM

In [19]:
df_matches

,similaridad,Unnamed: 0,DESCRIPCIÓN,PRESENTACION,CODIGO DE BARRAS,PRECIO CON IVA,PRECIO SIN IVA,PUBLICO\nSUGERIDO,descripcion_limpia,niprod,...,atrib1,atrib2,atrib3,atrib4,atrib5,atrib6,atrib7,atrib8,atrib9,descrip1_limpia
0,0.840433,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",8054083017068,"$ 740,145.59","$ 611,690.58","$ 1,073,211.02",HUMIRA AC 20mg,119551.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HUMIRA AC 20mg/2ml jerprell
1,0.766043,20005289,HUMIRA AC x 2 (40mg) HUMIRA AC x 1 (80 mg),"2 LAPICERAS PRELL. X 0,4 ml",8054083014050,"$ 1,480,292.66","$ 1,223,382.36","$ 2,146,424.35",HUMIRA AC 40mg HUMIRA AC,119552.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HUMIRA AC 40mg/4ml lapprell
2,0.352425,20021980,None,"1 LAPICERA PRELL. X 0,8 ml",8054083017648,"$ 1,553,789.84","$ 1,284,123.82","$ 2,252,995.41",,119569.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SYNAGIS
3,0.481004,20062091,RINVOQ 15MG RINVOQ 30MG,15 MG x 30 COMP REC DE LIB. PROLONGADA,8054083018706,"$ 1,799,606.71","$ 1,487,278.28","$ 2,609,429.58",RINVOQ 15MG RINVOQ 30MG,119559.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RINVOQ libprol
4,0.528795,20069005,RINVOQ 45MG,30 MG x 30 COMP REC DE LIB. PROLONGADA,8054083021386,"$ 3,184,521.02","$ 2,631,835.62","$ 4,617,555.55",RINVOQ 45MG,119551.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HUMIRA AC 20mg/2ml jerprell
5,0.352425,20078725,None,45 MG x 28 COMP REC DE LIB. PROLONGADA,8054083023922,"$ 4,494,517.30","$ 3,714,477.10","$ 6,517,049.97",,119569.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SYNAGIS
6,0.979447,20069017,SKYRIZI SKYRIZI,1 LAPICERA PRELLENADA por 1 ML (150 mg),8054083021935,"$ 4,344,159.74","$ 3,590,214.67","$ 6,299,031.64",SKYRIZI SKYRIZI,130998.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SKYRIZI
7,1.000000,20079636,SKYRIZI,360mg /2.4ml Cart. prellenado x 1 600mg /10ml ...,8054083023311,"$ 4,561,367.72","$ 3,769,725.40","$ 6,613,983.22",SKYRIZI,130998.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SKYRIZI
8,0.352425,20079635,None,None,8054083023328,"$ 4,561,367.72","$ 3,769,725.40","$ 6,613,983.22",,119569.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SYNAGIS
9,0.738401,20058578,KALETRA SOLUCIÓN,160 ML,8054083006406,"$ 148,177.93","$ 122,461.09","$ 214,857.98",KALETRA SOLUCIÓN,119557.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KALETRA
